# Ottimizazzione della funzione ***frag_faces***

In [1]:
using Pkg
Pkg.activate()
Pkg.instantiate()

  Activating project at `~/.julia/environments/v1.7`


In [2]:
#Carico le librerie utili all'implementazione ed alla fase di test
using Profile
using ProfileView
using BenchmarkTools #utile per il test
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

Testiamo la funzione merge_vertices originale

In [4]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

Preparazione input

In [5]:
V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);

Testiamo la funzione con l'utilizzo di @btime della libreria BenchmarkTools

In [6]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  191.250 μs (4786 allocations: 264.29 KiB)


([1.3260341 0.2707609000000001 0.24287710000000004; 1.3862088 0.21082410000000004 0.9148494000000003; 1.2660973 0.9422582000000003 0.30813880000000005; 1.3262719920385087 0.8823213992267827 0.9801110006439793], sparse([1, 3, 1, 4, 2, 3, 2, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, -1, 1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, -1, -1, 1], 1, 4))

# Miglioramento prestazioni

In [11]:
function frag_face_revisited(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    Threads.@threads for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

frag_face_revisited (generic function with 1 method)

In [16]:
Threads.nthreads()

1

In [14]:
@btime frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)

  196.125 μs (4888 allocations: 268.84 KiB)


([1.3260341 0.2707609000000001 0.24287710000000004; 1.3862088 0.21082410000000004 0.9148494000000003; 1.2660973 0.9422582000000003 0.30813880000000005; 1.3262719920385087 0.8823213992267827 0.9801110006439793], sparse([1, 3, 1, 4, 2, 3, 2, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, -1, 1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, -1, -1, 1], 1, 4))

In [9]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [10]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
  1╎1    @Base/Base.jl:35; getproperty(x::Module, f::Symbol)
  2╎2    @Base/Base.jl:42; getproperty(x::SparseVector{Int...
  1╎1    @Base/abstractarray.jl:2748; mapslices(f::LinearAlgebraicRep...
  1╎1    @Base/abstractarray.jl:2836; concatenate_setindex!(::Matrix{...
  1╎1    @Base/abstractarray.jl:1007; copyto_unaliased!(deststyle::In...
  1╎1    @Base/abstractarray.jl:1215; getindex(::Matrix{Float64}, ::I...
   ╎1    @Base/abstractarray.jl:2970; hash(A::Vector{Float64}, h::UIn...
  1╎ 1    @Base/float.jl:0; hash(x::Float64, h::UInt64)
   ╎1    @Base/abstractarray.jl:791; similar(a::Vector{Float64}, #un...
   ╎ 1    @Base/array.jl:378; similar
   ╎  1    @Base/boot.jl:466; Array
  1╎   1    @Base/boot.jl:457; Array
  1╎1    @Base/abstractset.jl:147; intersect!(s::Set{Any}, itr::Ve...
  2╎2    @Base/abstractset.jl:442; mapfilter(pred::Base.var"#84#85...
   ╎4    @Base/abstractset.jl:444; mapfilter(pred::Base.var"#84#85...
   ╎ 

   ╎    ╎    ╎    ╎ 1    @Base/boot.jl:467; Array
  1╎    ╎    ╎    ╎  1    @Base/boot.jl:459; Array
   ╎    ╎     4    In[7]:10; frag_face_revisited(V::Ma...
   ╎    ╎    ╎ 3    ...sparsematrix.jl:2541; getindex(A::SparseMatrix...
   ╎    ╎    ╎  3    ...sparsematrix.jl:2273; getindex_I_sorted(A::Spa...
   ╎    ╎    ╎   1    ...parsematrix.jl:2346; getindex_I_sorted_linea...
   ╎    ╎    ╎    1    @Base/array.jl:521; zeros
   ╎    ╎    ╎     1    @Base/array.jl:525; zeros
   ╎    ╎    ╎    ╎ 1    @Base/boot.jl:466; Array
  1╎    ╎    ╎    ╎  1    @Base/boot.jl:457; Array
  1╎    ╎    ╎   1    ...parsematrix.jl:2361; getindex_I_sorted_linea...
   ╎    ╎    ╎   1    ...parsematrix.jl:2383; getindex_I_sorted_linea...
  1╎    ╎    ╎    1    @Base/array.jl:861; getindex
   ╎    ╎    ╎ 1    ...sparsevector.jl:578; getindex
   ╎    ╎    ╎  1    ...sparsevector.jl:603; getindex(A::SparseMatrix...
  1╎    ╎    ╎   1    @Base/range.jl:837; iterate
  1╎    ╎     2    In[7]:11; frag_face_revisite

   ╎    ╎    ╎    ╎    ╎    1    @Base/set.jl:25; _Set
   ╎    ╎    ╎    ╎    ╎     1    @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎    ╎    ╎ 1    @Base/set.jl:6; Set
   ╎    ╎    ╎    ╎    ╎    ╎  1    ...e/dict.jl:90; Dict{Int64, Noth...
  1╎    ╎    ╎    ╎    ╎    ╎   1    ...e/boot.jl:457; Array
  1╎    ╎    ╎    ╎     1    ...actarray.jl:2769; mapslices(f::Linear...
  2╎    ╎    ╎    ╎     2    ...actarray.jl:2775; mapslices(f::Linear...
  2╎    ╎    ╎    ╎     4    ...actarray.jl:2785; mapslices(f::Linear...
   ╎    ╎    ╎    ╎    ╎ 2    ...reducedim.jl:15; reduced_indices(a::...
  2╎    ╎    ╎    ╎    ╎  2    ...educedim.jl:57; reduced_indices(ind...
  1╎    ╎    ╎    ╎     1    ...actarray.jl:2795; mapslices(f::Linear...
  3╎    ╎    ╎    ╎     3    ...actarray.jl:2797; mapslices(f::Linear...
  1╎    ╎    ╎    ╎     3    ...actarray.jl:2802; mapslices(f::Linear...
  2╎    ╎    ╎    ╎    ╎ 2    ...actarray.jl:2836; concatenate_setind...
  2╎    ╎    ╎     2    @Base/array.jl:729; 

   ╎    ╎    ╎    ╎     2    ...sevector.jl:1131; hcat(::Matrix{Float...
   ╎    ╎    ╎    ╎    ╎ 2    ...actarray.jl:1557; typed_hcat
   ╎    ╎    ╎    ╎    ╎  2    ...actarray.jl:1576; _typed_hcat(#unuse...
   ╎    ╎    ╎    ╎    ╎   2    ...actarray.jl:785; similar
   ╎    ╎    ╎    ╎    ╎    2    ...e/array.jl:378; similar
   ╎    ╎    ╎    ╎    ╎     2    @Base/boot.jl:467; Array
  2╎    ╎    ╎    ╎    ╎    ╎ 2    ...e/boot.jl:459; Array
  2╎    ╎    ╎    4    ...refactoring.jl:286; spaceindex(model::Tuple...
   ╎    ╎    ╎     2    ...refactoring.jl:214; coordintervals(coord::...
   ╎    ╎    ╎    ╎ 1    @Base/array.jl:126; vect
   ╎    ╎    ╎    ╎  1    @Base/array.jl:616; _array_for
   ╎    ╎    ╎    ╎   1    @Base/array.jl:613; _array_for
   ╎    ╎    ╎    ╎    1    ...ractarray.jl:828; similar
   ╎    ╎    ╎    ╎     1    ...ractarray.jl:829; similar
   ╎    ╎    ╎    ╎    ╎ 1    @Base/boot.jl:466; Array
  1╎    ╎    ╎    ╎    ╎  1    @Base/boot.jl:457; Array
   ╎    ╎    ╎  

   ╎    ╎    ╎    ╎    ╎   2    @Base/set.jl:25; _Set
   ╎    ╎    ╎    ╎    ╎    2    @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎    ╎     1    ...ractset.jl:97; union!(s::Set{Any...
   ╎    ╎    ╎    ╎    ╎    ╎ 1    @Base/set.jl:59; push!(s::Set{Any...
  1╎    ╎    ╎    ╎    ╎    ╎  1    ...e/dict.jl:382; setindex!(h::Di...
   ╎    ╎    ╎    ╎    ╎     1    ...ractset.jl:98; union!(s::Set{Any...
   ╎    ╎    ╎    ╎    ╎    ╎ 1    @Base/set.jl:57; length
   ╎    ╎    ╎    ╎    ╎    ╎  1    ...e/dict.jl:693; length
  1╎    ╎    ╎    ╎    ╎    ╎   1    ...e/Base.jl:42; getproperty
   ╎    ╎    ╎    ╎    ╎ 3    @Base/array.jl:2633; _shrink(shrinker!:...
   ╎    ╎    ╎    ╎    ╎  1    @Base/array.jl:2602; _shrink_filter!
  1╎    ╎    ╎    ╎    ╎   1    ...e/array.jl:2592; _unique_filter!
   ╎    ╎    ╎    ╎    ╎  2    @Base/array.jl:2628; vectorfilter(f::Fu...
   ╎    ╎    ╎    ╎    ╎   2    ...e/array.jl:2481; filter(f::Base.va...
  2╎    ╎    ╎    ╎    ╎    2    @Base/boot.jl:457; Array
 

...

   ╎    ╎    ╎    ╎   1    @Base/sort.jl:723; #sort!#8
   ╎    ╎    ╎    ╎    1    @Base/sort.jl:662; sort!
   ╎    ╎    ╎    ╎     1    @Base/sort.jl:591; sort!
   ╎    ╎    ╎    ╎    ╎ 1    @Base/sort.jl:592; sort!(v::Vector{Any...
  1╎    ╎    ╎    ╎    ╎  1    @Base/sort.jl:507; sort!(v::Vector{An...
   ╎    ╎    ╎    ╎ 6    ...mal_cycles.jl:136; (::LinearAlgebraicRepr...
   ╎    ╎    ╎    ╎  6    ...tractarray.jl:2849; map
   ╎    ╎    ╎    ╎   6    @Base/array.jl:653; collect_similar
   ╎    ╎    ╎    ╎    1    @Base/array.jl:744; _collect(c::Vector{A...
   ╎    ╎    ╎    ╎     1    ...generator.jl:44; iterate
   ╎    ╎    ╎    ╎    ╎ 1    @Base/array.jl:835; iterate
   ╎    ╎    ╎    ╎    ╎  1    @Base/array.jl:835; iterate
  1╎    ╎    ╎    ╎    ╎   1    @Base/array.jl:215; length
  4╎    ╎    ╎    ╎    5    @Base/array.jl:749; _collect(c::Vector{A...
   ╎    ╎    ╎    ╎     1    @Base/array.jl:607; _similar_for(c::Vec...
   ╎    ╎    ╎    ╎    ╎ 1    ...actarray.jl:791; simila

   ╎    ╎    ╎    ╎    ╎     2    @Base/boot.jl:474; Array
   ╎    ╎    ╎    ╎    ╎    ╎ 2    ...e/boot.jl:466; Array
  2╎    ╎    ╎    ╎    ╎    ╎  2    ...e/boot.jl:457; Array
  2╎    ╎    ╎     3    ...imal_cycles.jl:20; minimal_2cycles(V::Matr...
   ╎    ╎    ╎    ╎ 1    ...arsematrix.jl:687; convert(T::Type{Sparse...
   ╎    ╎    ╎    ╎  1    ...arsematrix.jl:676; SparseMatrixCSC
   ╎    ╎    ╎    ╎   1    ...rsematrix.jl:1092; copy
   ╎    ╎    ╎    ╎    1    ...rsematrix.jl:1081; ftranspose(A::Spars...
  1╎    ╎    ╎    ╎     1    @Base/boot.jl:457; Array
  2╎    ╎    ╎    95   ...arrangement.jl:412; componentgraph(V::Matri...
  1╎    ╎    ╎     1    ...arrangement.jl:0; get_external_cycle(V::...
   ╎    ╎    ╎     4    ...arrangement.jl:261; get_external_cycle(V::...
   ╎    ╎    ╎    ╎ 1    .../broadcast.jl:860; materialize
   ╎    ╎    ╎    ╎  1    ...erorderfns.jl:175; copy
   ╎    ╎    ╎    ╎   1    ...erorderfns.jl:170; _noshapecheck_map(::t...
   ╎    ╎    ╎    ╎    1    

   ╎    ╎    ╎    ╎   13   @Base/array.jl:2636; intersect
   ╎    ╎    ╎    ╎    8    @Base/array.jl:2632; _shrink(shrinker!::...
   ╎    ╎    ╎    ╎     3    ...stractset.jl:147; intersect!(s::Set{I...
   ╎    ╎    ╎    ╎    ╎ 3    @Base/set.jl:37; emptymutable
   ╎    ╎    ╎    ╎    ╎  3    @Base/set.jl:6; Set
   ╎    ╎    ╎    ╎    ╎   3    @Base/dict.jl:90; Dict{Int64, Nothin...
   ╎    ╎    ╎    ╎    ╎    1    ...e/array.jl:521; zeros
   ╎    ╎    ╎    ╎    ╎     1    ...e/array.jl:525; zeros
   ╎    ╎    ╎    ╎    ╎    ╎ 1    ...e/boot.jl:466; Array
  1╎    ╎    ╎    ╎    ╎    ╎  1    ...e/boot.jl:457; Array
  2╎    ╎    ╎    ╎    ╎    2    @Base/boot.jl:457; Array
   ╎    ╎    ╎    ╎     5    @Base/set.jl:23; Set
   ╎    ╎    ╎    ╎    ╎ 5    @Base/set.jl:25; _Set
   ╎    ╎    ╎    ╎    ╎  5    @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎    ╎   1    ...tractset.jl:95; union!(s::Set{Int6...
   ╎    ╎    ╎    ╎    ╎    1    @Base/set.jl:57; length
  1╎    ╎    ╎    ╎    ╎     1    @B

  1╎    ╎    ╎    ╎  1    @Base/boot.jl:457; Array
   ╎    ╎    ╎    53   ...arrangement.jl:423; componentgraph(V::Matri...
   ╎    ╎    ╎     35   ...c/utilities.jl:204; bbox(vertices::Matrix{...
  3╎    ╎    ╎    ╎ 35   ...tractarray.jl:2749; mapslices##kw
   ╎    ╎    ╎    ╎  3    ...tractarray.jl:2760; mapslices(f::LinearAl...
   ╎    ╎    ╎    ╎   3    @Base/array.jl:2637; setdiff
   ╎    ╎    ╎    ╎    3    @Base/array.jl:2632; _shrink(shrinker!::...
   ╎    ╎    ╎    ╎     3    @Base/set.jl:23; Set
   ╎    ╎    ╎    ╎    ╎ 3    @Base/set.jl:25; _Set
   ╎    ╎    ╎    ╎    ╎  3    @Base/set.jl:10; Set
   ╎    ╎    ╎    ╎    ╎   3    @Base/set.jl:6; Set
   ╎    ╎    ╎    ╎    ╎    3    @Base/dict.jl:90; Dict{Int64, Nothin...
   ╎    ╎    ╎    ╎    ╎     1    ...e/array.jl:521; zeros
   ╎    ╎    ╎    ╎    ╎    ╎ 1    .../array.jl:525; zeros
   ╎    ╎    ╎    ╎    ╎    ╎  1    ...e/boot.jl:466; Array
  1╎    ╎    ╎    ╎    ╎    ╎   1    ...e/boot.jl:457; Array
  2╎    ╎    ╎    ╎  

  1╎    ╎    ╎     1    @Base/array.jl:948; _growend!
   ╎    ╎     1    @Base/boot.jl:479; (Matrix)(x::Matrix{Float64})
   ╎    ╎    ╎ 1    @Base/array.jl:563; Array
   ╎    ╎    ╎  1    @Base/boot.jl:467; Array
  1╎    ╎    ╎   1    @Base/boot.jl:459; Array
   ╎1    @Base/task.jl:629; enq_work(t::Task)
  1╎ 1    @Base/task.jl:242; threadid
  1╎1    @Base/tuple.jl:89; indexed_iterate(t::Tuple{Int64,...
  1╎1    ...rc/IntervalTrees.jl:582; _push!(t::IntervalTrees.Interva...
  1╎1    ...rc/IntervalTrees.jl:166; insert!(leaf::IntervalTrees.Lea...
  1╎1    ...rc/IntervalTrees.jl:1304; nextintersection!(t::IntervalTr...
  1╎1    ...t/minimal_cycles.jl:187; (::LinearAlgebraicRepresentatio...
   ╎1    ...t/minimal_cycles.jl:9; (::LinearAlgebraicRepresentatio...
   ╎ 1    @Base/abstractarray.jl:1218; getindex
   ╎  1    ...multidimensional.jl:839; _getindex
   ╎   1    ...multidimensional.jl:851; _unsafe_getindex
   ╎    1    ...e/abstractarray.jl:783; similar
   ╎     1    @Base/array.jl:378